In [54]:
%load_ext autoreload
%autoreload 2

import sys  
sys.path.insert(0, 'code')

import pandas as pd
import xlsxwriter
import os
import os.path as osp
import numpy as np

#my code
from load_utils import loadExcel
from sheet_number_of_travels import createSheetNumberOfTravels
from sheet_times import createSheetTimes
from sheet_number_of_cars import createSheetNumberOfCars, computeData, createFakeDataset
import yaml

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
with open('parameters.yaml') as f:
    params = yaml.safe_load(f)

df_save = loadExcel('input/input.xlsx', N=params['number_of_cameras'])

In [66]:
df = df_save.copy(deep=True)

In [70]:
if osp.isfile('out.xlsx'):
    os.remove('out.xlsx')

with open('parameters.yaml') as f:
    params = yaml.safe_load(f)
    
try:
    writer = pd.ExcelWriter('out.xlsx', engine='xlsxwriter')
    createSheetNumberOfCars(df,writer,params)
    writer.close()
except Exception as e:
    print(e)
    writer.close()

In [50]:
res = computeData(df, params, startTime='00:00', endTime='23:59')

In [51]:
res['multiple']

Direction_to,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
Direction_from,,,,,,,,,,,,,,,,,,
1,27,56,982,32,1,5,1,10,5,77,6,103,0,52,0,3,4,15
2,231,8,120,2,49,0,2,1,7,14,2,19,0,9,10,3,7,0
3,5,60,110,1408,4,182,9,273,26,1688,8,101,3,35,20,112,69,822
4,257,1286,1155,53,181,3,34,16,44,79,16,78,10,57,38,18,42,18
5,5,20,13,287,71,941,11,403,12,552,5,12,2,13,12,181,23,81
6,69,2,207,13,1587,31,124,13,82,16,10,18,1,20,15,11,11,7
7,9,20,35,278,17,341,37,710,21,471,2,11,4,15,15,402,11,39
8,15,6,30,23,124,13,1031,71,99,28,61,9,4,21,8,35,3,5
9,89,87,278,1880,61,416,32,619,56,2577,18,119,8,80,29,390,14,123


In [58]:
# add fake data to force dataframe layout, aka have at least one entry for every possible combination
fake_df = createFakeDataset(params['number_of_cameras'])
df = df.append(fake_df, ignore_index=True)

# separate dataset into two parts based on license plate count
plate_counts = df["License_plate"].value_counts()
plate_counts['unknown'] = 0  # fake unknown values, so they are selected into single dataset
plate_count = plate_counts[plate_counts > 1]

df_multiple = df[df['License_plate'].isin(plate_count.index)]
df_single = df[~df['License_plate'].isin(plate_count.index)]

# create another dataset moved by one and merge it. Aka create pairs of all following directions
df_multiple = df_multiple.sort_values(by=['License_plate', 'Capture_time']).reset_index(drop=True)
df_multiple['joinIndex'] = df_multiple.index
df_multiple_moved = df_multiple.copy(deep=True)
df_multiple_moved['joinIndex'] = df_multiple_moved.index - 1
df_multiple_join = df_multiple.merge(df_multiple_moved, how='inner', on=['joinIndex', 'License_plate'],
                                     suffixes=['_from', '_to'])

In [63]:
df_multiple_join[(df_multiple_join['Direction_from'] == 13) & (df_multiple_join['Direction_to'] == 18)]

,License_plate,Capture_time_from,Direction_from,joinIndex,Capture_time_to,Direction_to


In [64]:
fake_df

,Capture_time,License_plate,Direction
0,2021-10-28 00:00:00,fake_1,1
1,2021-10-28 00:00:00,fake_1_1,1
2,2021-10-28 01:00:00,fake_1_1,1
3,2021-10-28 00:00:00,fake_1_2,1
4,2021-10-28 01:00:00,fake_1_2,2
...,...,...,...
625,2021-10-28 01:00:00,fake_18_15,15
626,2021-10-28 00:00:00,fake_18_16,18
627,2021-10-28 01:00:00,fake_18_16,16
628,2021-10-28 00:00:00,fake_18_17,18
